# Land Cover Over Time
How have land use and land cover changed over time?

In [1]:
import folium
import ee
import geemap
import geopandas as gpd
ee.Authenticate()

# Write your project ID here, in quotes
ee.Initialize(project = "emerge-lessons")

def add_ee_layer(self, ee_image_object, vis_params, name):
    """Adds a method for displaying Earth Engine image tiles to folium map."""
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
        name=name,
        overlay=True,
        control=True
    ).add_to(self)

folium.Map.add_ee_layer = add_ee_layer

In [2]:
map = folium.Map(location=[28.263363, -83.497652], tiles="Cartodb dark_matter", zoom_start=7)

In [3]:
landcover_1985 = (
    ee.ImageCollection('USFS/GTAC/LCMS/v2024-10')
      .filterDate('1985', '1986')
      .filter('study_area == "CONUS"')
      .first()
)

palette = ['fbff97', 'e6558b', '004e2b', '9dbac5', 'a6976a', '1b1716']
visual = {'min': 1, 'max': 6, 'palette': palette}

map.add_ee_layer(landcover_1985.select('Land_Use'), visual, '1985 Land Use')

In [4]:
display(map)

In [5]:
landcover_2024 = (
    ee.ImageCollection('USFS/GTAC/LCMS/v2024-10')
      .filterDate('2024', '2025')
      .filter('study_area == "CONUS"')
      .first()
)

map.add_ee_layer(landcover_2024.select('Land_Use'), visual, '2024 Land Use')
folium.LayerControl(collapsed = False).add_to(map)
display(map)

In the upper right, you can click the checkbox next to "2024 Land Use" to turn the layer on and off and compare the changes that occured in land use from 1985 to 2024.

Color Meanings ([from catalog](https://developers.google.com/earth-engine/datasets/catalog/USFS_GTAC_LCMS_v2024-10#bands))

$$\color{#fbff97}Agriculture$$
$$\color{#e6558b}Developed$$
$$\color{#004e2b}Forest$$
$$\color{#9dbac5}Other$$
$$\color{#a6976a}Rangeland or Pasture$$

Now, we will show land cover, as opposed to land use.

In [6]:
# Refresh the map
map = folium.Map(location=[28.263363, -83.497652], tiles="Cartodb dark_matter", zoom_start=7)

palette = ['004e2b', '009344', '61bb46', 'acbb67', '8b8560', 'cafd4b', 'f89a1c', '8fa55f', 'bebb8e', 'e5e98a', 'ddb925', '893f54', 'e4f5fd', '00b6f0', '1b1716']
visual = {'min': 1, 'max': 15, 'palette': palette}

# Add land cover for 1985 and 2024
map.add_ee_layer(landcover_1985.select('Land_Cover'), visual, '1985 Land Cover')
map.add_ee_layer(landcover_2024.select('Land_Cover'), visual, '2024 Land Cover')
folium.LayerControl(collapsed = False).add_to(map)
display(map)

$$\color{#004e2b}Trees$$
$$\color{#61bb46}Shrubs \space and \space Trees Mix$$
$$\color{#acbb67}Grass/Forb/Herb \space and \space Trees Mix$$
$$\color{#8b8560}Barren \space and \space Trees Mix$$
$$\color{#f89a1c}Shrubs$$
$$\color{#8fa55f}Grass/Forb/Herb \space and \space Shrubs Mix$$
$$\color{#bebb8e}Barren \space and \space Shrubs Mix$$
$$\color{#e5e98a}Grass/Forb/Herb$$
$$\color{#ddb925}Barren \space and \space Grass/Forb/Herb Mix$$
$$\color{#893f54}Barren or Impervious$$
$$\color{#e4f5fd}Snow or Ice$$

## Create Timelapse

In [7]:
landcover = (
    ee.ImageCollection('USFS/GTAC/LCMS/v2024-10')
      .filter('study_area == "CONUS"')
      .select('Land_Use')
)

point = ee.Geometry.Point(-81.660044, 28.473813)
region = point.buffer(distance=100000)

images = geemap.create_timeseries(
    landcover, '1985', '2024', region, frequency='year', reducer='first'
)
images

In [8]:
Map = geemap.Map()

labels = [str(y) for y in range(1985, 2024)]

Map.addLayer(images, visual, "Land Use")
Map.add_time_slider(images, visual, time_interval=2, labels=labels)
Map.setCenter(-81.660044, 28.473813, zoom=8)
Map

Map(center=[28.473813, -81.660044], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=Sea…

Click the > arrow to see a timelapse of the land use over the years. This may take a few seconds to load.

In [ ]:
fl = gpd.read_file('https://github.com/geo-di-lab/emerge-lessons/raw/refs/heads/main/docs/data/florida_boundary.geojson')[['geometry']]
region = geemap.geopandas_to_ee(fl)

timelapse = geemap.modis_ndvi_timelapse(
    region,
    out_gif='ndvi.gif',
    data='Terra',
    band='NDVI',
    start_date='2000-01-01',
    end_date='2024-12-31',
    frames_per_second=3,
    title='MODIS NDVI Timelapse',
)

Generating URL...
Please wait ...


In [ ]:
geemap.show_image(timelapse)

![Florida vegetation time lapse](https://github.com/geo-di-lab/emerge-lessons/blob/main/docs/ch4/ndvi.gif?raw=1)

References
- [Creating Timelapse Animations with geemap](https://book.geemap.org/chapters/09_timelapse.html)
- [Google Earth Engine Image Visualization](https://developers.google.com/earth-engine/guides/image_visualization#colab-python_2)